<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/w_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset.git
!pip install -qq transformers

Cloning into 'Ethos-Hate-Speech-Dataset'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 93 (delta 41), reused 52 (delta 13), pack-reused 0
Unpacking objects: 100% (93/93), done.
     |████████████████████████████████| 1.3MB 5.6MB/s 
     |████████████████████████████████| 2.9MB 21.1MB/s 
     |████████████████████████████████| 890kB 43.7MB/s 
     |████████████████████████████████| 1.1MB 46.3MB/s 


In [8]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [3]:
data_path='Ethos-Hate-Speech-Dataset/ethos/ethos_data/Ethos_Dataset_Binary.csv'
df = pd.read_csv(data_path, delimiter=';')
df['id'] = range(len(df))
df.set_index('id', inplace=True)
df.head()

,comment,isHate
id,,
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0


In [4]:
def to_category(score):
  if score <= 0.3:
    return 'NO'
  elif score>=0.7:
    return 'YES'
  else:
    return 'MID'

df['isHate_category']=df.isHate.apply(to_category)
df.head()

,comment,isHate,isHate_category
id,,,
0,You should know women's sports are a joke,1.0,YES
1,You look like Sloth with deeper Down’s syndrome,1.0,YES
2,You look like Russian and speak like Indian. B...,1.0,YES
3,"Women deserve to be abused, I guess.",1.0,YES
4,Women are made for making babies and cooking d...,1.0,YES


In [5]:
possible_labels=df.isHate_category.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'MID': 1, 'NO': 2, 'YES': 0}

In [6]:
df['label']=df.isHate_category.replace(label_dict)
df.head()

,comment,isHate,isHate_category,label
id,,,,
0,You should know women's sports are a joke,1.0,YES,0
1,You look like Sloth with deeper Down’s syndrome,1.0,YES,0
2,You look like Russian and speak like Indian. B...,1.0,YES,0
3,"Women deserve to be abused, I guess.",1.0,YES,0
4,Women are made for making babies and cooking d...,1.0,YES,0


In [9]:
RANDOM_SEED=17

X_train,X_val,y_train,y_val = train_test_split(
  df.index.values,
  df.label.values,
  test_size=0.2,
  random_state=RANDOM_SEED,
  stratify=df.label.values
)

In [10]:
df['data_type']=['not_set']*df.shape[0]
df.head()

,comment,isHate,isHate_category,label,data_type
id,,,,,
0,You should know women's sports are a joke,1.0,YES,0,not_set
1,You look like Sloth with deeper Down’s syndrome,1.0,YES,0,not_set
2,You look like Russian and speak like Indian. B...,1.0,YES,0,not_set
3,"Women deserve to be abused, I guess.",1.0,YES,0,not_set
4,Women are made for making babies and cooking d...,1.0,YES,0,not_set


In [11]:
df.loc[X_train,'data_type']='train'
# df.loc[df_val,'data_type']='val'
df.loc[X_val,'data_type']='val'

In [12]:
df.data_type.value_counts()

train    798
val      200
Name: data_type, dtype: int64

In [75]:
df.label.value_counts()

2    482
0    286
1    230
Name: label, dtype: int64

In [13]:
df.groupby(['isHate_category','label','data_type']).count()

comment  isHate
isHate_category label data_type                 
MID             1     train          184     184
                      val             46      46
NO              2     train          385     385
                      val             97      97
YES             0     train          229     229
                      val             57      57

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
import re
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import SnowballStemmer

from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer()  

def clean_comment(text, stops=True, stemming=True):
  text = str(text)
  text = text.lower().split()
  text = " ".join(text)
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"don't", "do not ", text)
  text = re.sub(r"aren't", "are not ", text)
  text = re.sub(r"isn't", "is not ", text)
  text = re.sub(r"%", " percent ", text)
  text = re.sub(r"that's", "that is ", text)
  text = re.sub(r"doesn't", "does not ", text)
  text = re.sub(r"he's", "he is ", text)
  text = re.sub(r"she's", "she is ", text)
  text = re.sub(r"it's", "it is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
  text = re.sub(r",", " ", text)
  text = re.sub(r"\.", " ", text)
  text = re.sub(r"!", " ! ", text)
  text = re.sub(r"\/", " ", text)
  text = re.sub(r"\^", " ^ ", text)
  text = re.sub(r"\+", " + ", text)
  text = re.sub(r"\-", " - ", text)
  text = re.sub(r"\=", " = ", text)
  text = re.sub(r"'", " ", text)
  text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
  text = re.sub(r":", " : ", text)
  text = re.sub(r" u s ", " american ", text)
  text = re.sub(r"\0s", "0", text)
  text = re.sub(r" 9 11 ", "911", text)
  text = re.sub(r"e - mail", "email", text)
  text = re.sub(r"j k", "jk", text)
  text = re.sub(r"\s{2,}", " ", text)
  text = text.lower().split()
  text = [w for w in text if len(w) >= 2]
  if stemming:
    englishStemmer = SnowballStemmer("english")
    text = [englishStemmer.stem(w) for w in text]
  if stops:
    text = [lemmatizer.lemmatize(w) for w in text if w not in stopwords.words('english')]
  text = " ".join(text)
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
clean_data=[]
for txt in df.comment:
  clean_data.append(clean_comment(txt, stops=True, stemming=True))
df.processed_comment=clean_data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [29]:
df['processed_comment']=df.comment.apply(clean_comment)

In [30]:
df.head()

,comment,isHate,isHate_category,label,data_type,processed_comment
id,,,,,,
0,You should know women's sports are a joke,1.0,YES,0,train,you should know women sports are joke
1,You look like Sloth with deeper Down’s syndrome,1.0,YES,0,train,you look like sloth with deeper down syndrome
2,You look like Russian and speak like Indian. B...,1.0,YES,0,val,you look like russian and speak like indian bo...
3,"Women deserve to be abused, I guess.",1.0,YES,0,train,women deserve to be abused guess
4,Women are made for making babies and cooking d...,1.0,YES,0,train,women are made for making babies and cooking d...


In [32]:
df['processed_comment2']=df.comment.apply(clean_comment)

In [33]:
df.head()

,comment,isHate,isHate_category,label,data_type,processed_comment,processed_comment2
id,,,,,,,
0,You should know women's sports are a joke,1.0,YES,0,train,you should know women sports are joke,know woman sport joke
1,You look like Sloth with deeper Down’s syndrome,1.0,YES,0,train,you look like sloth with deeper down syndrome,look like sloth deeper syndrom
2,You look like Russian and speak like Indian. B...,1.0,YES,0,val,you look like russian and speak like indian bo...,look like russian speak like indian disgust go...
3,"Women deserve to be abused, I guess.",1.0,YES,0,train,women deserve to be abused guess,woman deserv abus guess
4,Women are made for making babies and cooking d...,1.0,YES,0,train,women are made for making babies and cooking d...,woman made make babi cook dinner noth el


In [63]:
import collections
counts_Hate = {}
counts_Mid = {}
counts_NoHate = {}
counts={}
for row in df.itertuples():
  text = getattr(row, 'processed_comment2').split()
  for w in text:
    label=getattr(row, 'label')
    counts_Hate[w] = counts_Hate.get(w,0)
    counts_Mid[w] = counts_Mid.get(w,0)
    counts_NoHate[w] = counts_NoHate.get(w,0)
    if label==0:
      counts_Hate[w] = counts_Hate.get(w) + 1
    elif label==1:
      counts_Mid[w] = counts_Mid.get(w) + 1     
    elif label==2:
      counts_NoHate[w] = counts_NoHate.get(w) + 1
    counts[w] = counts.get(w,0) + 1



In [74]:
items = list(counts.items())
items.sort(key=lambda x:x[1],reverse=True)
print("word        hate mid nothate all".format(word,count))
for word,count in items:
  if count>10:
    print("{0:<10}{1:>5}{2:>5}{3:>7}{4:>5}".format(word,counts_Hate[word],counts_Mid[word],counts_NoHate[word],count))
    # print("{0:<10}{1:>5}".format(word,count))
    # print("{0:<10}{1:>5}".format(word,counts_Hate[word]))
    # print("{0:<10}{1:>5}".format(word,counts_Mid[word]))
    # print("{0:<10}{1:>5}".format(word,counts_NoHate[word]))

word        hate mid nothate all
peopl        51   45    109  205
like         33   46     55  134
fuck         66   26     34  126
woman        38   38     29  105
get          30   27     34   91
white        39   19     26   84
kill         36   22     11   69
black        31    9     25   65
would        19   13     33   65
becaus       19   16     29   64
one          13   13     37   63
know         14   10     36   60
look         13   13     30   56
muslim       35   12      9   56
right        21   11     23   55
want         23    9     22   54
make         15    8     30   53
go           15   13     23   51
whi          14   10     27   51
islam        33    8      8   49
love          4    8     36   48
countri      20   13     14   47
men          18   18     10   46
onli         15   12     19   46
shit         21    7     16   44
hate         17   11     16   44
think         5   15     23   43
take         19    9     13   41
kid           9    9     23   41
man       